# Phase 1 - Setup

In [1]:
!python --version
!nvidia-smi

Python 3.10.14
Tue Sep 10 13:58:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   39C    P8              1W /  140W |     211MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------

In [79]:
!pip list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Package                  Version
------------------------ ------------
accelerate               0.27.2
aiohappyeyeballs         2.4.0
aiohttp                  3.10.5
aiosignal                1.3.1
asttokens                2.4.1
async-timeout            4.0.3
attrs                    24.2.0
bitsandbytes             0.42.0
certifi                  2024.8.30
charset-normalizer       3.3.2
comm                     0.2.2
datasets                 2.18.0
debugpy                  1.8.5
decorator                5.1.1
dill                     0.3.8
docstring_parser         0.16
exceptiongroup           1.2.2
executing                2.1.0
filelock                 3.13.1
frozenlist               1.4.1
fsspec                   2024.2.0
huggingface-hub          0.24.6
idna                     3.8
ipykernel                6.29.5
ipython                  8.27.0
ipywidgets               8.1.5
jedi                     0.19.1
Jinja2                   3.1.3
jupyter_client           8.6.2
jupyter_core    

In [1]:
# install dependencies

%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

%pip install -q -U transformers==4.38.2
%pip install -q -U datasets==2.18.0
%pip install -q -U bitsandbytes==0.42.0
%pip install -q -U peft==0.9.0
%pip install -q -U trl==0.7.11
%pip install -q -U accelerate==0.27.2
%pip install --upgrade huggingface_hub

%pip install python-dotenv
%pip install ipywidgets


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 14.4 MB/

In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
# authen & author

from huggingface_hub import notebook_login
notebook_login()

In [12]:
from datasets import load_dataset

dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset['train'][0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [17]:
doc = dataset['train']['document'][0]
doc

'앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.'

In [21]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 글을 요약해주세요:\n\n앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.<end_of_turn>\n<start_of_turn>model\n'

In [22]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [23]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

**요약:**

* 앵커 정부는 수출 확대를 위해 총력을 기울이고 있으며, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다.
* 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다.
* 정부는 수출확대에 총력을 기울이기 위해 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다.


# Phase 2 - Train

In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [11]:
from datasets import load_dataset

data_files = {"train": "train.csv", "test": "test.csv"}
custom_dataset = load_dataset("csv", data_files=data_files)
custom_dataset

DatasetDict({
    train: Dataset({
        features: ['request', 'title', 'winner', 'description', 'publisher'],
        num_rows: 200
    })
    test: Dataset({
        features: ['request', 'title', 'winner', 'description', 'publisher'],
        num_rows: 4
    })
})

In [3]:
print(custom_dataset['train'][:1])
print(custom_dataset['test'][:1])

{'request': ['항상 사무실에서 가장 먼저 전화를 받는 사람에게 상을 주고 싶어.'], 'title': ['전화 응답왕상'], 'winner': ['김전화'], 'description': ['항상 전화가 울리기 무섭게 먼저 받아내는 그의 빠른 대처 능력을 칭찬하기 위해 이 상을 수여합니다.'], 'publisher': ['빠른 대응 전문가 협회']}
{'request': ['항상 책을 읽으며 점심 시간을 보내는 사람에게 상을 주고 싶어.'], 'title': ['점심 독서가상'], 'winner': ['이독서'], 'description': ['항상 점심 시간에 책을 읽으며 지식을 쌓는 그의 열정을 칭찬하기 위해 이 상을 수여합니다.'], 'publisher': ['사내 독서 장려회']}


In [4]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['title'])):
        prompt_list.append(
            f"""<bos><start_of_turn>user
다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:

한국어로 작성해.

[요청]
{example['request'][i]}<end_of_turn>
<start_of_turn>model
{{'title':'{example['title'][i]}', 'winner':'{example['winner'][i]}', 'description':'{example['description'][i]}', 'publisher':'{example['publisher'][i]}'}}<end_of_turn><eos>"""
)
    return prompt_list

In [5]:
train_data = custom_dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:

한국어로 작성해.

[요청]
항상 사무실에서 가장 먼저 전화를 받는 사람에게 상을 주고 싶어.<end_of_turn>
<start_of_turn>model
{'title':'전화 응답왕상', 'winner':'김전화', 'description':'항상 전화가 울리기 무섭게 먼저 받아내는 그의 빠른 대처 능력을 칭찬하기 위해 이 상을 수여합니다.', 'publisher':'빠른 대응 전문가 협회'}<end_of_turn><eos>


In [6]:
# Hugging Face 로그인

from dotenv import load_dotenv
import os
from huggingface_hub import login

# load .env
load_dotenv()
HUGGINGFACE_ACCESS_TOKEN = os.environ.get('HUGGINGFACE_ACCESS_TOKEN')


login(HUGGINGFACE_ACCESS_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/jiuk/.cache/huggingface/token
Login successful


In [7]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [8]:
BASE_MODEL = "google/gemma-2b-it"
ADAPTER_MODEL = "lora_adapter"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

print(BASE_MODEL)
print(ADAPTER_MODEL)
print(device)
print(model)

/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


google/gemma-2b-it
lora_adapter
cuda:0
GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_att

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        # num_train_epochs = 1,
        max_steps=10,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [10]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=10, training_loss=4.180945587158203, metrics={'train_runtime': 13.9008, 'train_samples_per_second': 2.878, 'train_steps_per_second': 0.719, 'total_flos': 67160679825408.0, 'train_loss': 4.180945587158203, 'epoch': 0.2})

In [11]:
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
pipe_finetuned

In [12]:
prompt = '<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어로 작성해.\n[요청]\n돈까스를 잘먹는 내 친구에게 상을 주고 싶어.<end_of_turn>\n<start_of_turn>model\n'
prompt

'<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어로 작성해.\n[요청]\n돈까스를 잘먹는 내 친구에게 상을 주고 싶어.<end_of_turn>\n<start_of_turn>model\n'

In [13]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

**돈까스의 맛과 즐거움을 위한 추천**

안녕하세요, 친구.

저는 당신에게 돈까스를 추천해 드립니다. 돈까스는 우리나 한국의 전통 음식으로, 맛과 풍부한 칼로리의 조화로운 음식입니다.

저는 당신이 돈까스를 잘 먹는 데 도움이 되는 사람이라고 생각하며, 당신에게 추천해 드립니다. 

저는 당신이 돈까스를 즐기는 데 도움이 되는 사람이라고 생각하며, 당신에게 추천해 드립니다.


In [14]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [15]:
!ls -alh lora_adapter

total 29M
drwxr-xr-x 2 jiuk jiuk 4.0K Sep 10 21:10 .
drwxr-xr-x 7 jiuk jiuk 4.0K Sep 10 21:13 ..
-rw-r--r-- 1 jiuk jiuk 5.0K Sep 10 21:20 README.md
-rw-r--r-- 1 jiuk jiuk  690 Sep 10 21:20 adapter_config.json
-rw-r--r-- 1 jiuk jiuk  29M Sep 10 21:20 adapter_model.safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
BASE_MODEL = "google/gemma-2b-it"
# ADAPTER_MODEL = "lora_adapter"
ADAPTER_MODEL = "v3"

# base_model과 new_model에 저장된 LoRA 가중치를 통합하여 새로운 모델을 생성
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    low_cpu_mem_usage=True,
    return_dict=True,
    device_map='auto',
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(
    base_model, 
    ADAPTER_MODEL, 
    low_cpu_mem_usage=True,
    device_map='auto', 
    torch_dtype=torch.float16
    ) # LoRA 가중치를 가져와 기본 모델에 통합


model = model.merge_and_unload()
# model.save_pretrained('gemma-2b-it-award-factory')

/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model.save_pretrained('gemma-2b-it-award-factory')

In [1]:
!ls -alh ./gemma-2b-it-award-factory

total 4.7G
drwxr-xr-x 2 jiuk jiuk 4.0K Sep 10 19:20 .
drwxr-xr-x 9 jiuk jiuk 4.0K Sep 10 19:17 ..
-rw-r--r-- 1 jiuk jiuk  662 Sep 10 19:17 config.json
-rw-r--r-- 1 jiuk jiuk  132 Sep 10 19:17 generation_config.json
-rw-r--r-- 1 jiuk jiuk 4.7G Sep 10 19:20 model-00001-of-00002.safetensors
-rw-r--r-- 1 jiuk jiuk  65M Sep 10 19:20 model-00002-of-00002.safetensors
-rw-r--r-- 1 jiuk jiuk  14K Sep 10 19:20 model.safetensors.index.json


# Phase 3 - Eval

In [2]:
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-award-factory-ko-v1"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jiuk/.pyenv/versions/3.10.14/envs/award-factory/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [4]:
prompt = '<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어로 작성해.\n[요청]\n우리 동네를 순찰하는 강아지 호두에게 감사한 마음을 담아서 상을 주고 싶어.<end_of_turn>\n<start_of_turn>model\n'
prompt

'<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어로 작성해.\n[요청]\n우리 동네를 순찰하는 강아지 호두에게 감사한 마음을 담아서 상을 주고 싶어.<end_of_turn>\n<start_of_turn>model\n'

In [5]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

NameError: name 'doc' is not defined

In [6]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

{'title':'순찰왕상', 'winner':'최호두', 'description':'우리 동네를 순찰하며 사람들의 생을 지켜주는 그의 성실함을 칭찬하기 위해 이 상을 수여합니다.', 'publisher':'강아지 순찰 협회'}


In [16]:
custom_dataset['test'][:1]

{'request': ['항상 책을 읽으며 점심 시간을 보내는 사람에게 상을 주고 싶어.'],
 'title': ['점심 독서가상'],
 'winner': ['이독서'],
 'description': ['항상 점심 시간에 책을 읽으며 지식을 쌓는 그의 열정을 칭찬하기 위해 이 상을 수여합니다.'],
 'publisher': ['사내 독서 장려회']}

In [24]:
prompt = '<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어로 작성해.\n[요청]\n가장 많은 동아리 활동에 참여한 직원에게 상을 주고 싶어.<end_of_turn>\n<start_of_turn>model\n'
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

{'title':'동아리 마스터상', 'winner':'이동아', 'description':'다양한 동아리에서 활동하며 창의력과 열정을 보여주는 그의 노력을 칭찬하기 위해 이 상을 수여합니다.', 'publisher':'동아리 활동 협회'}


In [7]:
prompt = '<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어 단어만 사용할 것. 영어를 사용하면 안돼.\n[요청]\n항상 가족에게 새로운 요리를 선보이는 사람에게 상을 주고 싶어.<end_of_turn>\n<start_of_turn>model\n'
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

{'title':'요리 선보기 전문가상', 'winner':'최요리', 'description':'가족에게 새로운 및 delish한 요리를 선보며 추억을 만드는 그의 요리 실력을 칭찬하기 위해 이 상을 수여합니다.', 'publisher':'가족 요리 전문가 협회'}


In [9]:
prompt = '<bos><start_of_turn>user\n다음 요청을 고려해서 아래의 예시처럼 상장 문구를 작성해주세요:\n한국어 단어만 사용할 것. 영어를 사용하면 안돼.\n[요청]\n항상 새로운 소프트웨어를 개발하는 개발자에게 상을 주고 싶어<end_of_turn>\n<start_of_turn>model\n'
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

{'title':'코드 마법사상', 'winner':'최개발', 'description':'혁신적인 소프트웨어를 개발해 세상을 변화시키는 그의 뛰어난 개발 능력을 칭찬하기 위해 이 상을 수여합니다.', 'publisher':'소프트웨어 개발 협회'}


# Phase 4 - upload

In [3]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your m

In [6]:
FINETUNE_MODEL = "gemma-2b-it-award-factory-ko-v1"

finetune_model.push_to_hub(FINETUNE_MODEL, use_temp_dir=False)
tokenizer.push_to_hub(FINETUNE_MODEL, use_temp_dir=False)